# **Additional (Library, Comment, etc)**

In [ ]:
import pandas as pd
import numpy as np
import random as ra
from google.colab import files, drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Initialization**

**Bobot & Bias**

In [ ]:
def weight(n_input,n_hidden):
  hidden = []
  for i in range(n_hidden):
    hidden.append([])
    for j in range(n_input):
      rand = ra.uniform(-0.5,0.5)
      hidden[i].append(rand)
  return hidden

In [ ]:
def bias(n_hidden):
  hidden = []
  for i in range(n_hidden):
    rand = ra.uniform(-0.5,0.5)
    hidden.append(rand)
  return hidden

# **Inside the Model**

**Normalisasi**

In [ ]:
def normalisasi(data):
  data = data.astype(float)
  dataMax = data[0][0]
  dataMin = data[0][0]
  for i in data:
    if (dataMax < np.max(i)):
      dataMax = np.max(i)
    elif (dataMin > np.min(i)):
      dataMin = np.min(i)
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j] = (data[i][j] - dataMin)/(dataMax - dataMin)
  return data,dataMin,dataMax

**Split Dataset**

In [ ]:
def splittingData(data,split):
  split = int(split * len(data))
  train, test = data[:split], data[split:]
  return train,test

**Training**

In [ ]:
def sigmoid(in_hidden):
  for i in range(len(in_hidden)):
    for j in range(len(in_hidden[i])):
      in_hidden[i][j] = 1.0/(1.0 + np.exp(-in_hidden[i][j]))
  return in_hidden

In [ ]:
def moorePenrose(out_hidden):
  hPlus = np.dot(np.linalg.inv(np.dot(out_hidden.T,out_hidden)),out_hidden.T)
  return hPlus

In [ ]:
def outputWeight(hPlus,y_train):
  out_weight = np.dot(hPlus,y_train)
  return out_weight

In [ ]:
def outputHidden(x_train,tw_hidden,b_hidden):
  in_hidden = np.dot(x_train,tw_hidden)
  for i in range(len(in_hidden)):
    for j in range(len(in_hidden[0])):
      in_hidden[i][j]= in_hidden[i][j] + b_hidden[j]
  out_hidden = sigmoid(in_hidden)
  return in_hidden,out_hidden

In [ ]:
def training(x_train,y_train,w_hidden,b_hidden):
  in_hidden,out_hidden = outputHidden(x_train,np.transpose(w_hidden),b_hidden)
  hPlus = moorePenrose(out_hidden)
  out_weight = outputWeight(hPlus,y_train)
  return out_weight

**Testing**

In [ ]:
def testing(x_test,y_test,w_hidden,b_hidden,out_weight):
  in_hidden,out_hidden = outputHidden(x_test,np.transpose(w_hidden),b_hidden)
  predict = np.dot(out_hidden,out_weight)
  return predict

**Denormalisasi (AMAN)**

In [ ]:
def denormalisasi(data,minimum,maksimum):
  if (isinstance(data[0], list) == False) :
      data = [[i] for i in data]
  hasil = []
  for i in range(len(data)):
    value = (data[i][0]*(maksimum-minimum))+minimum
    hasil.append(value)
  return hasil

**RMSE (AMAN)**

In [ ]:
def evaluasi(aktual,prediksi):
  hasil = 0
  for i in range(len(aktual)):
    hasil += (aktual[i]-prediksi[i])**2
  hasil = (hasil/len(prediksi))**.5
  return hasil

# **Main**

In [ ]:
dataset = pd.read_excel('/content/gdrive/My Drive/Skripsi/Dataset/Real/DRD.xlsx',sheet_name='6 Fitur')
dataset,dataMin,dataMax = normalisasi(dataset[dataset.columns[:7]].values)
ra.shuffle(dataset)
nor_data = dataset[:,:6]
nor_target = dataset[:,6]

split = 0.8
x_train,x_test = splittingData(nor_data,split)
y_train,y_test = splittingData(nor_target,split)

for i in range(10):
  n_input = len(x_train[0])
  n_hidden = 5
  w_hidden = weight(n_input,n_hidden)
  b_hidden = bias(n_hidden)

  out_weight = training(x_train,y_train,w_hidden,b_hidden)
  predict = testing(x_test,y_test,w_hidden,b_hidden,out_weight)
  denor_target_predict = denormalisasi(predict,dataMin,dataMax)
  denor_target_aktual = denormalisasi(y_test,dataMin,dataMax)
  eval = evaluasi(denor_target_aktual,denor_target_predict)
  print(eval)

21358.415593339687
14934.064901264574
13979.599678115428
15441.814109412995
20390.271498281312
12882.273036700877
16674.77926060919
19747.107219940903
16164.809951221265
12802.27779790136
